In [16]:
# import packages for analysis and modeling
import pandas as pd  # data frame operations
#from pandas.tools.plotting import scatter_matrix  # scatter plot matrix
import statsmodels.api as sm  # statistical models (including regression)
import statsmodels.formula.api as smf  # R-like model specification
import numpy as np  # arrays and math functions
import matplotlib.pyplot as plt  # 2D plotting
#import zipcodes
#import pyzipcode
import time

import warnings
warnings.filterwarnings("ignore")

from pandas import Series
from scipy.stats import uniform  # for training-and-test split
from scipy.stats import boxcox
from sklearn.tree import DecisionTreeRegressor  # machine learning tree
from sklearn.ensemble import RandomForestRegressor # ensemble method
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from pprint import pprint
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARIMAResults
from math import sqrt

import warnings
warnings.filterwarnings("ignore")

# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return diff


# invert differenced value
def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]

In [17]:
house = pd.read_csv('clean_house_data.csv', encoding = "ISO-8859-1")

states = house.State.unique()
#states = states[:10]
#states = states[10:20] # failed on OH
#states = states[20:30]
#states = states[30:40]
#states = states[40:]

#states = states[28:29] #only AR
#states = states[6:7]
#states = np.append(states[0:6], states[7:]) # filter OH
#print(states[16])

states = np.append(states[0:16], states[17:]) # filter OH

#'''
df = pd.DataFrame(columns=['State', 'Zip', 'Date', 'Prediction'])
dfRMSE = pd.DataFrame(columns=['Zip', 'RMSE'])

start_time = time.time()
count = 0

for s in states:
    
    ar = house.loc[house.State == s]
    ar = ar.drop(ar.loc[:,ar.columns[6:15]], axis=1)
    ar = ar.drop(ar.loc[:,ar.columns[258:]], axis=1)
    
    ar_validate = house.loc[house.State == s]
    ar_validate = ar_validate.drop(ar_validate.loc[:,ar_validate.columns[6:267]], axis=1)
    
    zips = ar.RegionName.unique()

    #zips = zips[:1] # for testing
    
    training = pd.melt(ar, id_vars=['RegionName'], value_vars=ar.columns[6:])
    testing = pd.melt(ar_validate, id_vars=['RegionName'], value_vars=ar_validate.columns[6:])
    
    #print(ar.columns)
    count = count+1
    print('/************************************************************************************************/')
    print('Processing state {}: {}'.format(s, count))
    print("--- %s minutes ---" % ((time.time() - start_time)/60))
    
    for z in zips:
        train_region = training.loc[training.RegionName == z]
        test_region = testing.loc[testing.RegionName == z]
        
        X = train_region.value.astype('float32').values
        Y = test_region.value.astype('float32').values
        D = test_region.variable.values
        
        train, test, date = X, Y, D
        # walk-forward validation
        history = [x for x in train]
        predictions = list()
        
        #print('Prediction for State: {}, Zip: {}'.format(s, str(z)))
        
        for i in range(len(test)):
            # difference data
            months = 12
            diff = difference(history, months)
            # predict
            model = ARIMA(diff, order=(2,1,0))
            model_fit = model.fit(trend='nc', disp=0)
            yhat = model_fit.forecast()[0]
            #print(yhat)
            yhat = inverse_difference(history, yhat, months)
            predictions.append(yhat)
            # observation 
            history.append(test[i])
            #print('Prediction for Date: {} : ${}'.format(date[i], round(float(yhat),2)))
            #print('>Predicted=%.3f, Expected=%3.f' % (yhat, date[i]))
            df = df.append({'State': s,
                            'Zip': z,
                            'Date':date[i],
                            'Prediction':round(float(yhat),0)
                           }, ignore_index=True)
            
        # report performance
        mse = mean_squared_error(test, predictions)
        rmse = sqrt(mse)
        #print('RMSE: %.3f' % rmse)
        dfRMSE = dfRMSE.append({'ZIP': z, 'RMSE': rmse}, ignore_index=True)
        
df.to_csv('us_zip_analysis_complete.csv', index = False)
dfRMSE.to_csv('us_zip_rmse.csv', index = False)

#df.to_csv('us_zip_analysis.csv', mode = 'a', header=False, index = False)

print("--- %s minutes ---" % ((time.time() - start_time)/60))
#'''

/************************************************************************************************/
Processing state IL: 1
--- 0.0003490606943766276 minutes ---
/************************************************************************************************/
Processing state TX: 2
--- 3.6229817310969037 minutes ---
/************************************************************************************************/
Processing state NY: 3
--- 9.349839782714843 minutes ---
/************************************************************************************************/
Processing state CA: 4
--- 16.179243369897208 minutes ---
/************************************************************************************************/
Processing state TN: 5
--- 25.590911614894868 minutes ---
/************************************************************************************************/
Processing state FL: 6
--- 28.223292477925618 minutes ---
/*******************************************************